# Модель BERT и GPT

Взять датасет https://huggingface.co/datasets/merionum/ru_paraphraser решить задачу парафраза

(дополнительно необязательная задача)на выбор взять https://huggingface.co/datasets/sberquad https://huggingface.co/datasets/blinoff/medical_qa_ru_data натренировать любую модель для вопросно ответной системы как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER

## Загрузим данные с помощью библиотеки datasets

In [1]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 9.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer
     

In [4]:
from datasets import load_dataset

corpus = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-62b622e3b2274f4e/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [6]:
corpus['train']['text_1'][5]

'Вертолет с 11 иностранцами на борту упал в Пакистане'

In [7]:
corpus['train']['text_2'][5]

'В Пакистане упал вертолет с 11 иностранцами'

In [8]:
corpus['train']['class'][5]

'1'

Загрузка предобученной модели

In [9]:
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

Функция парафраза

In [11]:
result = []

def paraphrase(row):

  text_1 = corpus['train']['text_1'][row]
  text_2 = corpus['train']['text_2'][row]

  batch = tokenizer(text_1, text_2, return_tensors='pt').to(model.device)
  with torch.inference_mode():
      prob = torch.softmax(model(**batch).logits, -1).numpy()

  result.append([text_1, text_2, prob[0][1]])

  return pd.DataFrame(result, columns=['Текст 1', 'Текст 2', 'prob'])

In [12]:
for row in range(100):
  res = paraphrase(row)
     

In [13]:
res

,Текст 1,Текст 2,prob
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,0.841314
2,Агент: РФС вновь задерживает зарплату Фабио Ка...,СМИ: Агент Фабио Капелло грозится подать в суд...,0.120681
3,День Победы в Москве обещает выдаться облачным,Любляна отпразднует День Победы вместе с Москвой,0.053447
4,Посол РФ в США: Россия будет бороться с попытк...,Правительство запланировало заработать на лоте...,0.053032
...,...,...,...
95,День Победы в режиме онлайн,Парад в честь Дня Победы проходит в Донецке,0.072619
96,В Индии боевики-маоисты захватили 200 человек,В Индии маоисты захватили в заложники 500 человек,0.967957
97,Боевики-маоисты взяли в заложники 200 человек ...,В Индии боевики-маоисты захватили 200 человек,0.981236
98,Боевики-маоисты взяли в заложники 200 человек ...,В Индии маоисты захватили в заложники 500 человек,0.977859
